In [ ]:
import ee
import datetime

# Authenticate and initialize Google Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project='ee-steviaanlenaa')

# Define Jakarta Selatan as the region of interest
jakarta_selatan = ee.Geometry.Point([106.8227, -6.3000])  # Approximate center

# Define time range (last 6 months for better data coverage)
end_date = datetime.datetime.today().strftime('%Y-%m-%d')
start_date = (datetime.datetime.today() - datetime.timedelta(days=180)).strftime('%Y-%m-%d')

# Load SMAP Soil Moisture dataset (0-5cm surface moisture)
smap = ee.ImageCollection("NASA_USDA/HSL/SMAP10KM_soil_moisture") \
    .filterDate(start_date, end_date) \
    .select("ssm")  # "ssm" = Surface Soil Moisture

# Check if data is available
if smap.size().getInfo() == 0:
    print("⚠️ No recent SMAP soil moisture data available. Using MODIS NDVI instead.")
    use_ndvi = True
    mean_soil_moisture = None
else:
    mean_soil_moisture = smap.mean()
    use_ndvi = False

# Load long-term average soil moisture (2015-2023)
historical_avg = ee.ImageCollection("NASA_USDA/HSL/SMAP10KM_soil_moisture") \
    .filterDate("2015-01-01", "2023-12-31") \
    .select("ssm")

# Ensure historical data is valid
if historical_avg.size().getInfo() == 0:
    print("⚠️ No historical SMAP data available. Cannot calculate soil moisture anomaly.")
    historical_avg = None
else:
    historical_avg = historical_avg.mean()

# Compute Soil Moisture Anomaly (if both datasets exist)
if mean_soil_moisture and historical_avg:
    soil_moisture_anomaly = mean_soil_moisture.subtract(historical_avg)

    # Get the soil moisture anomaly at Jakarta Selatan
    result = soil_moisture_anomaly.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=jakarta_selatan,
        scale=10000,
        bestEffort=True
    )

    print("✅ Soil Moisture Anomaly in Jakarta Selatan:", result.getInfo())

# --- ALTERNATIVE: Use MODIS NDVI for Vegetation Dryness ---
if use_ndvi:
    ndvi_dataset = ee.ImageCollection("MODIS/006/MOD13A2") \
        .filterDate(start_date, end_date) \
        .select("NDVI")  # NDVI (Normalized Difference Vegetation Index)

    if ndvi_dataset.size().getInfo() == 0:
        print("⚠️ No MODIS NDVI data available. Cannot analyze vegetation drought.")
    else:
        ndvi_mean = ndvi_dataset.mean()
        ndvi_result = ndvi_mean.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=jakarta_selatan,
            scale=500,
            bestEffort=True
        )
        print("🌿 NDVI Value (Vegetation Health) in Jakarta Selatan:", ndvi_result.getInfo())


⚠️ No recent SMAP soil moisture data available. Using MODIS NDVI instead.


/usr/local/lib/python3.11/dist-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for MODIS/006/MOD13A2! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD13A2

  warnings.warn(warning, category=DeprecationWarning)


⚠️ No MODIS NDVI data available. Cannot analyze vegetation drought.


In [ ]:
import ee
import datetime

# Authenticate and initialize Google Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project='ee-steviaanlenaa')

# Define Jakarta Selatan as the region of interest
jakarta_selatan = ee.Geometry.Point([106.8227, -6.3000])  # Approximate center

# Define time range (last 12 months for more data)
end_date = datetime.datetime.today().strftime('%Y-%m-%d')
start_date = (datetime.datetime.today() - datetime.timedelta(days=365)).strftime('%Y-%m-%d')

# 📌 1️⃣ MODIS NDVI (Vegetation Health Indicator)
try:
    ndvi_dataset = ee.ImageCollection("MODIS/061/MOD13A2")  # Updated dataset
    ndvi = ndvi_dataset.filterDate(start_date, end_date).select("NDVI").mean()
    ndvi_value = ndvi.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=jakarta_selatan,
        scale=500,
        bestEffort=True
    )
    print("🌿 NDVI (Vegetation Health) in Jakarta Selatan:", ndvi_value.getInfo())

except Exception as e:
    print("⚠️ No MODIS NDVI data available:", str(e))

# 📌 2️⃣ CHIRPS Rainfall Anomaly (Drought Indicator)
try:
    chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").filterDate(start_date, end_date).select("precipitation")
    avg_rainfall = chirps.mean().reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=jakarta_selatan,
        scale=5000,
        bestEffort=True
    )
    print("🌧️ Average Precipitation in Jakarta Selatan:", avg_rainfall.getInfo())

except Exception as e:
    print("⚠️ No CHIRPS rainfall data available:", str(e))

# 📌 3️⃣ Sentinel-1 SAR Moisture (Backup for Soil Moisture)
try:
    sentinel1 = ee.ImageCollection("COPERNICUS/S1_GRD") \
        .filterDate(start_date, end_date) \
        .filterBounds(jakarta_selatan) \
        .select("VV")  # VV band detects surface wetness

    avg_sar = sentinel1.mean().reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=jakarta_selatan,
        scale=500,
        bestEffort=True
    )
    print("🛰️ Sentinel-1 Surface Moisture in Jakarta Selatan:", avg_sar.getInfo())

except Exception as e:
    print("⚠️ No Sentinel-1 moisture data available:", str(e))


🌿 NDVI (Vegetation Health) in Jakarta Selatan: {'NDVI': 3499.7272727272725}
🌧️ Average Precipitation in Jakarta Selatan: {'precipitation': 7.44556188583374}
🛰️ Sentinel-1 Surface Moisture in Jakarta Selatan: {'VV': -2.983412719457825}


In [ ]:
import ee
import datetime

# Authenticate and initialize Google Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project='ee-steviaanlenaa')

# Define Jakarta Selatan as the region of interest
jakarta_selatan = ee.Geometry.Point([106.8227, -6.3000])  # Approximate center

# Define time range (last 12 months for more data)
end_date = datetime.datetime.today().strftime('%Y-%m-%d')
start_date = (datetime.datetime.today() - datetime.timedelta(days=365)).strftime('%Y-%m-%d')

# 📌 1️⃣ MODIS NDVI (Vegetation Health Indicator)
try:
    ndvi_dataset = ee.ImageCollection("MODIS/061/MOD13A2")  # Updated dataset
    ndvi = ndvi_dataset.filterDate(start_date, end_date).select("NDVI").mean()
    ndvi_value = ndvi.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=jakarta_selatan,
        scale=500,
        bestEffort=True
    )
    print("🌿 NDVI (Vegetation Health) in Jakarta Selatan:", ndvi_value.getInfo())

except Exception as e:
    print("⚠️ No MODIS NDVI data available:", str(e))

# 📌 2️⃣ CHIRPS Rainfall Anomaly (Drought Indicator)
try:
    chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").filterDate(start_date, end_date).select("precipitation")
    avg_rainfall = chirps.mean().reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=jakarta_selatan,
        scale=5000,
        bestEffort=True
    )
    print("🌧️ Average Precipitation in Jakarta Selatan:", avg_rainfall.getInfo())

except Exception as e:
    print("⚠️ No CHIRPS rainfall data available:", str(e))

# 📌 3️⃣ Sentinel-1 SAR Moisture (Backup for Soil Moisture)
try:
    sentinel1 = ee.ImageCollection("COPERNICUS/S1_GRD") \
        .filterDate(start_date, end_date) \
        .filterBounds(jakarta_selatan) \
        .select("VV")  # VV band detects surface wetness

    avg_sar = sentinel1.mean().reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=jakarta_selatan,
        scale=500,
        bestEffort=True
    )
    print("🛰️ Sentinel-1 Surface Moisture in Jakarta Selatan:", avg_sar.getInfo())

except Exception as e:
    print("⚠️ No Sentinel-1 moisture data available:", str(e))


In [ ]:
import ee
import datetime

# Authenticate and initialize Google Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project='ee-steviaanlenaa')

# Load Indonesia administrative boundaries (GADM Level 2: Kabupaten/Kota)
gadm = ee.FeatureCollection("FAO/GAUL/2015/level2").filter(ee.Filter.eq("ADM0_NAME", "Indonesia"))

# Define time range (last 12 months)
end_date = datetime.datetime.today().strftime('%Y-%m-%d')
start_date = (datetime.datetime.today() - datetime.timedelta(days=365)).strftime('%Y-%m-%d')

# Function to calculate NDVI, Rainfall, and Soil Moisture for each region
def analyze_region(region):
    try:
        # NDVI (Vegetation Health)
        ndvi_dataset = ee.ImageCollection("MODIS/061/MOD13A2").filterDate(start_date, end_date).select("NDVI")
        ndvi = ndvi_dataset.mean().reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=region.geometry(),
            scale=5000,
            bestEffort=True
        ).get("NDVI")

        # CHIRPS Rainfall
        chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").filterDate(start_date, end_date).select("precipitation")
        rainfall = chirps.mean().reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=region.geometry(),
            scale=5000,
            bestEffort=True
        ).get("precipitation")

        # Sentinel-1 SAR Moisture
        sentinel1 = ee.ImageCollection("COPERNICUS/S1_GRD")\
            .filterDate(start_date, end_date)\
            .filterBounds(region.geometry())\
            .select("VV")  # VV band for surface wetness

        moisture = sentinel1.mean().reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=region.geometry(),
            scale=5000,
            bestEffort=True
        ).get("VV")

        return {
            "kabupaten_kota": region.get("ADM2_NAME").getInfo(),
            "🌿NDVI": ndvi.getInfo() if ndvi else None,
            "🌧️Rainfall": rainfall.getInfo() if rainfall else None,
            "🛰️Moisture": moisture.getInfo() if moisture else None
        }
    except Exception as e:
        return None

# Analyze all Kabupaten/Kota
results = []
for region in gadm.toList(gadm.size()).getInfo():
    region_feature = ee.Feature(region)
    data = analyze_region(region_feature)
    if data:
        results.append(data)

# Rank top 3 based on NDVI + Rainfall + Moisture
ranked = sorted(results, key=lambda x: (x.get("NDVI", 0)) + (x.get("Rainfall", 0)) + (x.get("Moisture", 0)), reverse=True)

top_3 = ranked[:3]

# Print Top 3 Most Potential Areas
print("Top 3 Most Potential Areas for Green Projects in Indonesia:")
for i, area in enumerate(top_3, 1):
    print(f"{i}. {area['kabupaten_kota']} - 🌿NDVI: {area['NDVI']}, 🌧️Rainfall: {area['Rainfall']}, 🛰️Moisture: {area['Moisture']}")

Top 3 Most Potential Areas for Green Projects in Indonesia:


KeyError: 'NDVI'

#Infrastruktur hijau

In [ ]:
import requests
from bs4 import BeautifulSoup
import random

# Daftar provinsi di Indonesia
provinces = [
    "Aceh", "Sumatera Utara", "Sumatera Barat", "Riau", "Jambi", "Sumatera Selatan", "Bengkulu", "Lampung", "Bangka Belitung", "Kepulauan Riau",
    "DKI Jakarta", "Jawa Barat", "Jawa Tengah", "DI Yogyakarta", "Jawa Timur", "Banten",
    "Bali", "Nusa Tenggara Barat", "Nusa Tenggara Timur",
    "Kalimantan Barat", "Kalimantan Tengah", "Kalimantan Selatan", "Kalimantan Timur", "Kalimantan Utara",
    "Sulawesi Utara", "Sulawesi Tengah", "Sulawesi Selatan", "Sulawesi Tenggara", "Gorontalo", "Sulawesi Barat",
    "Maluku", "Maluku Utara", "Papua", "Papua Barat", "Papua Selatan", "Papua Tengah", "Papua Pegunungan"
]

# Daftar kategori infrastruktur hijau
infra_types = [
    "Taman Kota", "Hutan Kota", "Roof Garden", "Biopori", "Penampungan Air Hujan", "Jalur Hijau", "Solar Panel",
    "Bangunan Hemat Energi", "Dinding Hijau", "Ekowisata", "Mangrove Reforestation", "Biofuel Plantations",
    "Pertanian Vertikal", "Rekayasa Air Limbah Hijau", "Transportasi Berkelanjutan"
]

# Coba melakukan web scraping
url = "https://www.mongabay.co.id/tag/infrastruktur-hijau/"
try:
    response = requests.get(url, timeout=10)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all("h2", class_="post-title")  # Ambil judul artikel

    if not articles:
        raise Exception("Tidak ada artikel yang ditemukan")
except Exception as e:
    print("Gagal mengakses situs, gunakan data random.")
    articles = []

# Menentukan 1 kategori infrastruktur hijau per provinsi
green_projects = {}
for province in provinces:
    if articles:
        infra_type = random.choice(articles).text.strip()
    else:
        infra_type = random.choice(infra_types)
    green_projects[province] = infra_type

# Cetak hasil
for province, infra in green_projects.items():
    print(f"{province}: {infra}")

Gagal mengakses situs, gunakan data random.
Aceh: Roof Garden
Sumatera Utara: Mangrove Reforestation
Sumatera Barat: Penampungan Air Hujan
Riau: Mangrove Reforestation
Jambi: Bangunan Hemat Energi
Sumatera Selatan: Penampungan Air Hujan
Bengkulu: Transportasi Berkelanjutan
Lampung: Bangunan Hemat Energi
Bangka Belitung: Biopori
Kepulauan Riau: Ekowisata
DKI Jakarta: Roof Garden
Jawa Barat: Bangunan Hemat Energi
Jawa Tengah: Transportasi Berkelanjutan
DI Yogyakarta: Hutan Kota
Jawa Timur: Ekowisata
Banten: Roof Garden
Bali: Dinding Hijau
Nusa Tenggara Barat: Solar Panel
Nusa Tenggara Timur: Penampungan Air Hujan
Kalimantan Barat: Rekayasa Air Limbah Hijau
Kalimantan Tengah: Dinding Hijau
Kalimantan Selatan: Penampungan Air Hujan
Kalimantan Timur: Biopori
Kalimantan Utara: Solar Panel
Sulawesi Utara: Solar Panel
Sulawesi Tengah: Hutan Kota
Sulawesi Selatan: Solar Panel
Sulawesi Tenggara: Bangunan Hemat Energi
Gorontalo: Dinding Hijau
Sulawesi Barat: Jalur Hijau
Maluku: Rekayasa Air Limba

In [ ]:
import requests
from bs4 import BeautifulSoup
import random

# Daftar provinsi di Indonesia
provinces = [
    "Aceh", "Sumatera Utara", "Sumatera Barat", "Riau", "Jambi", "Sumatera Selatan", "Bengkulu", "Lampung",
    "Bangka Belitung", "Kepulauan Riau", "DKI Jakarta", "Jawa Barat", "Jawa Tengah", "DI Yogyakarta", "Jawa Timur", "Banten",
    "Bali", "Nusa Tenggara Barat", "Nusa Tenggara Timur", "Kalimantan Barat", "Kalimantan Tengah", "Kalimantan Selatan",
    "Kalimantan Timur", "Kalimantan Utara", "Sulawesi Utara", "Sulawesi Tengah", "Sulawesi Selatan", "Sulawesi Tenggara",
    "Gorontalo", "Sulawesi Barat", "Maluku", "Maluku Utara", "Papua", "Papua Barat", "Papua Selatan", "Papua Tengah", "Papua Pegunungan"
]

# Kategori Infrastruktur Hijau
green_infra = [
    "Roof Garden", "Mangrove Reforestation", "Penampungan Air Hujan", "Bangunan Hemat Energi", "Transportasi Berkelanjutan",
    "Biopori", "Ekowisata", "Hutan Kota", "Dinding Hijau", "Solar Panel", "Rekayasa Air Limbah Hijau", "Jalur Hijau", "Biofuel Plantations"
]

# Situs berita untuk scraping
news_sites = [
    "https://www.kompas.com/tag/infrastruktur-hijau",
    "https://www.detik.com/tag/infrastruktur-hijau"
]

def scrape_news(site):
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}  # Menghindari deteksi bot
        response = requests.get(site, headers=headers, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            articles = soup.find_all('h2')  # Bisa disesuaikan berdasarkan struktur HTML
            return [article.text.strip() for article in articles]
        else:
            print(f"⚠️ Gagal mengakses {site}, status code: {response.status_code}")
            return []
    except Exception as e:
        print(f"⚠️ Error saat scraping {site}: {str(e)}")
        return []

# Coba scraping dari dua situs berita
articles = []
for site in news_sites:
    articles.extend(scrape_news(site))

# Jika gagal scraping, gunakan data random
results = {}
for province in provinces:
    if articles:
        infra_type = random.choice(articles)  # Ambil satu artikel secara random
    else:
        infra_type = random.choice(green_infra)  # Pakai data random
    results[province] = infra_type

# Tampilkan hasil
for province, infra in results.items():
    print(f"{province}: {infra}")


Aceh: Makassar-Australia Berkongsi, Bangun Infrastruktur Hijau di Permukiman Kumuh
Sumatera Utara: Sejauh Mana Komitmen Pemerintah Membangun Infrastruktur Hijau?
Sumatera Barat: Jokowi Harap Jepang Tingkatkan Kontribusi untuk Konektivitas dan Infrastruktur Hijau ASEAN
Riau: Akhirnya, Sisa 9 Kilometer Tol Pekanbaru-Bangkinang Tuntas Dibangun
Jambi: Ini Manfaat Infrastruktur Hijau di Perkotaan, Tingkatkan Kualitas Hidup sampai Perekonomian
Sumatera Selatan: Sejauh Mana Komitmen Pemerintah Membangun Infrastruktur Hijau?
Bengkulu: Komitmen Pemerintah Bangun Jalan Tol yang Mendukung Infrastruktur Hijau
Lampung: Jokowi Ajak ExxonMobil Investasi di Bidang Energi Baru Terbarukan dan Infrastruktur Hijau
Bangka Belitung: Komitmen Pemerintah Bangun Jalan Tol yang Mendukung Infrastruktur Hijau
Kepulauan Riau: KTT G20 dan Peningkatan Kerja Sama Infrastruktur
DKI Jakarta: Komitmen Pemerintah Bangun Jalan Tol yang Mendukung Infrastruktur Hijau
Jawa Barat: Komitmen Pemerintah Bangun Jalan Tol yang Men

In [ ]:
import requests
from bs4 import BeautifulSoup
import random

# Daftar provinsi di Indonesia
provinces = [
    "Aceh", "Sumatera Utara", "Sumatera Barat", "Riau", "Jambi", "Sumatera Selatan", "Bengkulu", "Lampung",
    "Bangka Belitung", "Kepulauan Riau", "DKI Jakarta", "Jawa Barat", "Jawa Tengah", "DI Yogyakarta", "Jawa Timur", "Banten",
    "Bali", "Nusa Tenggara Barat", "Nusa Tenggara Timur", "Kalimantan Barat", "Kalimantan Tengah", "Kalimantan Selatan",
    "Kalimantan Timur", "Kalimantan Utara", "Sulawesi Utara", "Sulawesi Tengah", "Sulawesi Selatan", "Sulawesi Tenggara",
    "Gorontalo", "Sulawesi Barat", "Maluku", "Maluku Utara", "Papua", "Papua Barat", "Papua Selatan", "Papua Tengah", "Papua Pegunungan"
]

# Kategori Infrastruktur Hijau dan kata kunci yang berhubungan
green_infra_mapping = {
    "Roof Garden": ["roof garden", "atap hijau"],
    "Mangrove Reforestation": ["mangrove", "reforestasi"],
    "Penampungan Air Hujan": ["air hujan", "penampungan air"],
    "Bangunan Hemat Energi": ["hemat energi", "bangunan hijau"],
    "Transportasi Berkelanjutan": ["transportasi berkelanjutan", "kendaraan listrik"],
    "Biopori": ["biopori"],
    "Ekowisata": ["ekowisata", "wisata hijau"],
    "Hutan Kota": ["hutan kota"],
    "Dinding Hijau": ["dinding hijau", "vertical garden"],
    "Solar Panel": ["solar panel", "energi surya"],
    "Rekayasa Air Limbah Hijau": ["air limbah", "pengolahan limbah"],
    "Jalur Hijau": ["jalur hijau", "jalan hijau"],
    "Biofuel Plantations": ["biofuel", "energi biomassa"]
}

# Situs berita untuk scraping
news_sites = [
    "https://www.kompas.com/tag/infrastruktur-hijau",
    "https://www.detik.com/tag/infrastruktur-hijau"
]

def scrape_news(site):
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}  # Menghindari deteksi bot
        response = requests.get(site, headers=headers, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            articles = soup.find_all('h2')  # Bisa disesuaikan berdasarkan struktur HTML
            return [article.text.strip().lower() for article in articles]
        else:
            print(f"⚠️ Gagal mengakses {site}, status code: {response.status_code}")
            return []
    except Exception as e:
        print(f"⚠️ Error saat scraping {site}: {str(e)}")
        return []

# Coba scraping dari dua situs berita
articles = []
for site in news_sites:
    articles.extend(scrape_news(site))

# Mapping artikel ke kategori infrastruktur hijau
def get_category(title):
    for category, keywords in green_infra_mapping.items():
        if any(keyword in title for keyword in keywords):
            return category
    return random.choice(list(green_infra_mapping.keys()))  # Default jika tidak cocok

# Menentukan kategori per provinsi
results = {}
for province in provinces:
    if articles:
        infra_type = get_category(random.choice(articles))  # Coba dari artikel
    else:
        infra_type = random.choice(list(green_infra_mapping.keys()))  # Gunakan data random
    results[province] = infra_type

# Tampilkan hasil
for province, infra in results.items():
    print(f"{province}: {infra}")


Aceh: Solar Panel
Sumatera Utara: Bangunan Hemat Energi
Sumatera Barat: Jalur Hijau
Riau: Biopori
Jambi: Dinding Hijau
Sumatera Selatan: Rekayasa Air Limbah Hijau
Bengkulu: Roof Garden
Lampung: Jalur Hijau
Bangka Belitung: Solar Panel
Kepulauan Riau: Hutan Kota
DKI Jakarta: Penampungan Air Hujan
Jawa Barat: Bangunan Hemat Energi
Jawa Tengah: Ekowisata
DI Yogyakarta: Ekowisata
Jawa Timur: Jalur Hijau
Banten: Solar Panel
Bali: Hutan Kota
Nusa Tenggara Barat: Transportasi Berkelanjutan
Nusa Tenggara Timur: Jalur Hijau
Kalimantan Barat: Roof Garden
Kalimantan Tengah: Penampungan Air Hujan
Kalimantan Selatan: Biopori
Kalimantan Timur: Bangunan Hemat Energi
Kalimantan Utara: Biopori
Sulawesi Utara: Transportasi Berkelanjutan
Sulawesi Tengah: Transportasi Berkelanjutan
Sulawesi Selatan: Bangunan Hemat Energi
Sulawesi Tenggara: Bangunan Hemat Energi
Gorontalo: Rekayasa Air Limbah Hijau
Sulawesi Barat: Biofuel Plantations
Maluku: Biofuel Plantations
Maluku Utara: Ekowisata
Papua: Ekowisata
Papu

In [ ]:
import pandas as pd
import numpy as np
import ee
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Load dataset
df = pd.read_excel("indeks2024.xlsx")
df.head()


,Provinsi,Poverty_Index,Latitude,Longitude,Unnamed: 4
0,ACEH,12.64,4.695135,96.749399,NaN
1,SUMATERA UTARA,7.19,2.115355,99.545097,NaN
2,SUMATERA BARAT,5.42,-0.739940,100.800005,NaN
3,RIAU,6.36,0.293347,101.706829,NaN
4,JAMBI,7.26,-1.485183,102.438058,NaN


In [ ]:
df.drop(columns=["Provinsi", "Unnamed: 4"], inplace=True)
df.head()

,Poverty_Index,Latitude,Longitude
0,12.64,4.695135,96.749399
1,7.19,2.115355,99.545097
2,5.42,-0.739940,100.800005
3,6.36,0.293347,101.706829
4,7.26,-1.485183,102.438058


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Poverty_Index  37 non-null     float64
 1   Latitude       37 non-null     float64
 2   Longitude      37 non-null     float64
dtypes: float64(3)
memory usage: 1020.0 bytes


In [ ]:
# Initialize Google Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project='ee-steviaanlenaa')

# Function to fetch geospatial data
def fetch_geospatial_data(lat, lon):
    point = ee.Geometry.Point(lon, lat).buffer(10000)  # 10km buffer
    end_date = ee.Date("2024-01-01")
    start_date = end_date.advance(-60, "day")  # 60-day period

    try:
        # Nighttime Lights (VIIRS)
        viirs = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG")\
            .filterDate(start_date, end_date)\
            .select("avg_rad")\
            .mean()\
            .reduceRegion(reducer=ee.Reducer.mean(), geometry=point, scale=500, bestEffort=True)
        night_lights = viirs.getInfo().get("avg_rad", np.nan)

        # Daylight Duration (ERA5)
        solar = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")\
            .filterDate(start_date, end_date)\
            .select("surface_solar_radiation_downwards")\
            .mean()\
            .reduceRegion(reducer=ee.Reducer.mean(), geometry=point, scale=500, bestEffort=True)
        daylight_duration = solar.getInfo().get("surface_solar_radiation_downwards", np.nan)

        print(f"Lat: {lat}, Lon: {lon} -> Night Lights: {night_lights}, Daylight Duration: {daylight_duration}")
        return night_lights, daylight_duration
    except Exception as e:
        print("Error fetching geospatial data:", str(e))
        return np.nan, np.nan

df["Night_Lights"], df["Daylight_Duration"] = zip(*df.apply(lambda row: fetch_geospatial_data(row["Latitude"], row["Longitude"]), axis=1))

df = df.dropna()

if df.empty:
    raise ValueError("No data left after preprocessing. Check geospatial data fetching and input file.")
df.head()

Lat: 4.695135, Lon: 96.7493993 -> Night Lights: 0.37557116888730824, Daylight Duration: 10509469.674845738
Lat: 2.1153547, Lon: 99.5450974 -> Night Lights: 0.12988677413354932, Daylight Duration: 10585868.050452534
Lat: -0.7399397, Lon: 100.8000051 -> Night Lights: 0.03242087295689579, Daylight Duration: 11882843.98647012
Lat: 0.2933469, Lon: 101.7068294 -> Night Lights: 0.42811377868869477, Daylight Duration: 11641271.160402158
Lat: -1.4851831, Lon: 102.4380581 -> Night Lights: 0.6239315647825784, Daylight Duration: 13368051.431123504
Lat: -3.3194374, Lon: 103.914399 -> Night Lights: 1.0225577550791833, Daylight Duration: 14636658.083862498
Lat: -3.5778471, Lon: 102.3463875 -> Night Lights: 0.30415367416605554, Daylight Duration: 14439241.504615933
Lat: -4.5585849, Lon: 105.4068079 -> Night Lights: 0.6221974780903287, Daylight Duration: 14949560.326478936
Lat: -2.7410513, Lon: 106.4405872 -> Night Lights: 0.2775935957257204, Daylight Duration: 14980872.867282959
Lat: 3.9456514, Lon: 1

,Poverty_Index,Latitude,Longitude,Night_Lights,Daylight_Duration
0,12.64,4.695135,96.749399,0.375571,1.050947e+07
1,7.19,2.115355,99.545097,0.129887,1.058587e+07
2,5.42,-0.739940,100.800005,0.032421,1.188284e+07
3,6.36,0.293347,101.706829,0.428114,1.164127e+07
4,7.26,-1.485183,102.438058,0.623932,1.336805e+07


In [ ]:
# Features and target
X = df[["Night_Lights", "Daylight_Duration"]]
y = df["Poverty_Index"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

# Save model
with open("poverty_model.pkl", "wb") as f:
    pickle.dump(model, f)

print("Model saved as poverty_model.pkl")

Mean Absolute Error: 4.535949999999998
Model saved as poverty_model.pkl


In [ ]:
print(f"Min Poverty Index: {df['Poverty_Index'].min()}, Max Poverty Index: {df['Poverty_Index'].max()}")

Min Poverty Index: 3.8, Max Poverty Index: 29.66
